# 🧪 Lab 2: Geospatial AI with the Prithvi Foundation Model

**Objective:** Learn to use a powerful, pre-trained geospatial foundation model (`Prithvi-100M-sen1floods11`) to perform a real-world analysis: **flood segmentation**.

**Why this is important:** This lab demonstrates how to run state-of-the-art AI models *directly within a notebook* using only the **CPU**. This "local inference" workflow is a key skill, allowing you to quickly test models and analyze data without complex, expensive GPU infrastructure.

### 🧠 What is Prithvi?

**Prithvi** (Sanskrit for "Earth") is a new class of **Geospatial Foundation Model (GFM)** developed by IBM and NASA. Unlike traditional AI models that are trained on one specific task (like *only* floods or *only* burn scars), Prithvi was pre-trained on a massive, diverse dataset of satellite imagery (from the HLS dataset) from across the entire United States.

This "foundation" training gives it a deep, generalized understanding of what land, water, vegetation, and urban areas look like in different seasons and conditions. We can then "fine-tune" this base model for specific tasks. The model we are using today, `Prithvi-100M-sen1floods11`, has been fine-tuned to be an expert at one thing: **identifying water**.

We will run the entire process—from finding data to visualizing the AI's prediction—right here in this notebook.

---

## Key Concepts

  * **Foundation Model (FM):** A large AI model (like Prithvi) pre-trained on vast amounts of general data. This "foundation" allows it to be easily adapted to new, specific tasks.
  * **Inference:** The process of *using* a trained model to make predictions on new data. This is what we are doing today. (The opposite is *training*).
  * **STAC (SpatioTemporal Asset Catalog):** A modern "search engine" or API for geospatial data. We use it to find the exact satellite images we need from cloud providers like the Microsoft Planetary Computer.
  * **Sentinel-2:** An advanced Earth observation mission providing high-resolution (10m) optical imagery. Its multiple **spectral bands** (seeing beyond just Red, Green, and Blue) are perfect for AI.
  * **GeoTIFF:** A standard file format for satellite images. It's a "geospatial" TIFF, meaning it contains crucial metadata like coordinates (latitude/longitude) and the map projection.
  * **TerraTorch:** An open-source library used to easily load and work with geospatial foundation models like Prithvi.

# 1. ⚙️ Verify Your Environment

This first code cell is a critical check. It **verifies that you are running the correct Jupyter kernel** (`geo-labs-lab2`).

**Why we do this:** All the specialized libraries for this lab (like `terratorch`, `pystac_client`, and `mmseg`) have been pre-installed into a specific environment. If you run this notebook with the default kernel, the code in the later steps will fail.

* **If you see `✅ Correct kernel`**, you are all set! Move to the next step.
* **If you see `⚠️ WARNING`**, please follow the instructions printed in the output to change your kernel.

If numpy < 2 is installed no need to run

import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy<2", "--force-reinstall", "--no-deps"])

In [ ]:
import sys
import os

# Check if we're in the correct environment
current_env = os.path.basename(sys.prefix)
expected_env = "geo-labs-lab2"

if current_env == expected_env:
    print(f"✅ Correct kernel: {current_env}")
    print("All packages are pre-installed via the setup script.")
else:
    print(f"⚠️  WARNING: Wrong kernel detected!")
    print(f"   Current: {current_env}")
    print(f"   Expected: {expected_env}")
    print()
    print("Please change your kernel:")
    print("   1. Click 'Kernel' → 'Change Kernel'")
    print("   2. Select 'Python (geo-labs-lab2)'")
    print("   3. Re-run this cell")
    raise RuntimeError(f"Wrong kernel: {current_env}. Please select '{expected_env}'")

# 2\. 📚 Import Libraries

Now that we've confirmed our environment, this cell will **import all the specific Python libraries** we need. We are loading tools for several key tasks, grouped by their function:

  * **AI & Deep Learning:** `torch` (PyTorch) is the core deep learning framework. `mmseg`, `mmcv`, and `terratorch` are helper libraries specifically for loading and running segmentation models like Prithvi.
  * **Geospatial Data:** `rasterio`, `geopandas`, and `gdal` are the industry-standard tools for opening, handling, and reprojecting satellite images (GeoTIFFs) and vector data (like our AOI).
  * **Data Search:** `pystac_client` and `planetary_computer` allow us to connect to and search the STAC catalog.
  * **Visualization:** `leafmap` provides the interactive map for our final result.
  * **Standard Utilities:** `requests`, `numpy`, `os`, and `time` are used for downloading files, numerical operations, and timing our model.

In [ ]:
import requests
import leafmap
import numpy as np
import torch
import rasterio
import rasterio.warp
import shutil
import time
import imageio.v2 as imageio
from skimage.transform import resize

# Visualization libraries
from IPython.display import HTML, display
from ipyleaflet import Marker, Popup
import ipywidgets as widgets

# STAC & Geospatial Libraries
import pystac_client
import planetary_computer
import geopandas as gpd
from shapely.geometry import box, shape

# GDAL
from osgeo import gdal

import mmcv
import mmseg

# AI Model Libraries
from huggingface_hub import hf_hub_download
from terratorch.models import EncoderDecoderFactory

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"MMCV version: {mmcv.__version__}")
print(f"MMSeg version: {mmseg.__version__}")

# 3\. 🛰️ Find Data with STAC (Sentinel-2)

Before we can run our model, we need data. We can't just "Google" for satellite images; we need a machine-readable way to find analysis-ready data. This is where **STAC (SpatioTemporal Asset Catalog)** comes in.

This code block will:

1.  **Define an Area of Interest (AOI):** We've chosen coordinates over **Vicksburg, Mississippi**. This area is a classic example of a complex river system (the Mississippi and Yazoo Rivers) with surrounding floodplains, making it a great test for our model.
2.  **Connect to a STAC Catalog:** We'll connect to the **Microsoft Planetary Computer**, a massive, open catalog of geospatial data.
3.  **Search for Data:** We will search for a `sentinel-2-l2a` (Level-2A, analysis-ready) image that intersects our AOI, was taken during the Spring 2023 flood season, and has **low cloud cover** (`"lt": 30`). Seeing the ground is essential\!

In [ ]:
# 1. Define Area of Interest - Mississippi River near Vicksburg (inland floods)
min_lon, min_lat, max_lon, max_lat = [-91.2, 32.2, -90.8, 32.5]
aoi_geometry = box(min_lon, min_lat, max_lon, max_lat)
aoi_gdf = gpd.GeoDataFrame(geometry=[aoi_geometry], crs="EPSG:4326")

# 2. Connect to Planetary Computer STAC
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
print("🛰️ Connected to Planetary Computer STAC catalog.")

# 3. Search for Sentinel-2 data during flood season
time_of_interest = "2023-04-01/2023-05-31"  # Spring flood season
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=aoi_gdf.geometry[0],
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 30}}
)

items = list(search.items())
if len(items) == 0:
    raise Exception("No Sentinel-2 items found. Check your internet connection.")

first_item = items[0]
print(f"✅ Found {len(items)} items. Selected: {first_item.id}")

# 4\. ⬇️ Download the Data

Our STAC search has *found* a matching Sentinel-2 scene. This cell will **download the specific band files** we need.

**Why these bands?** A "true color" image (like the `rendered_preview` or your phone's camera) only has 3 bands: Red, Green, and Blue. A Sentinel-2 scene is **multispectral**—it captures light in many different wavelengths, including those invisible to the human eye.

The `Prithvi-100M-sen1floods11` model was specifically trained on **6 bands** to be extra-perceptive:

  * `B02` (Blue), `B03` (Green), `B04` (Red) - Visible light
  * `B08` (Near-Infrared / NIR) - Key for seeing vegetation health.
  * `B11` (SWIR1), `B12` (SWIR2) - **Short-Wave Infrared**. These bands are *excellent* at detecting moisture and distinguishing water from land, which is why they are critical for a flood model.

We will also download the `rendered_preview` (a simple JPG) to use as a "True Color" background image on our final map for comparison.

In [ ]:
# Sentinel-2 band names for Prithvi model
required_assets = {
    "B02": "Blue.tif",     # Blue (10m)
    "B03": "Green.tif",    # Green (10m)
    "B04": "Red.tif",      # Red (10m)
    "B08": "Nir.tif",      # NIR (10m)
    "B11": "Swir1.tif",    # SWIR1 (20m)
    "B12": "Swir2.tif",    # SWIR2 (20m)
}

data_dir = "hls_mississippi_data"
os.makedirs(data_dir, exist_ok=True)

# Clean old data to avoid conflicts with previous runs
if os.path.exists(data_dir):
    try:
        shutil.rmtree(data_dir)
        print(f"♻️  Refreshing data directory...")
    except PermissionError:
        print(f"⚠️  Data directory in use - will overwrite files instead")
    except Exception as e:
        print(f"⚠️  Note: {e}")

os.makedirs(data_dir, exist_ok=True)

# Helper function to download files
def download_file(url, folder, filename):
    filepath = os.path.join(folder, filename)
    if not os.path.exists(filepath):
        print(f"⬇️ Downloading {filename}...")
        r = requests.get(url, stream=True)
        r.raise_for_status()
        with open(filepath, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    else:
        print(f"✅ {filename} already exists.")
    return filepath

# Loop and download all required files
band_filepaths = []
true_color_path = ""
for asset_name, filename in required_assets.items():
    try:
        href = first_item.assets[asset_name].href
        fpath = download_file(href, data_dir, filename)
        if asset_name == 'browse':
            true_color_path = fpath
        else:
            # Add the 6 model bands to a list
            band_filepaths.append(fpath)
    except KeyError:
        print(f"⚠️ Warning: Asset '{asset_name}' not found in item. Skipping.")
    except Exception as e:
        print(f"❌ Error downloading {asset_name}: {e}")

# Separately download the true color image for visualization
true_color_path = ""
try:
    href = first_item.assets["rendered_preview"].href
    true_color_path = download_file(href, data_dir, "True_Color.jpg")
    print("✓ Downloaded true color preview for visualization")
except:
    print("⚠️ True color preview not available")

print(f"\n✅ Downloaded {len(band_filepaths)} bands for model inference")
print("Contents:", os.listdir(data_dir))

Great! Your `hls_mississippi_data` folder should now be populated with the GeoTIFF files for all 6 bands, plus the `True_Color.jpg` preview. We are ready to run the AI.

# 5\. 🧠 Load AI Model & Run Inference

This is the core of the lab. The following cell performs the entire AI analysis. Here is a breakdown of what it's doing:

1.  **Define Helper Functions:**

      * `normalize_and_stack`: This is crucial. Our 6 Sentinel-2 bands have different resolutions (10m and 20m). This function will **reproject** all bands to match a single reference grid, **stack** them into one 6-layer file, and **normalize** the pixel values. Normalization (scaling values to a standard range) is a required step to prepare data for an AI model.
      * `resize_tensor`: The original satellite image is *huge* (over 10,000x10,000 pixels). Running the model on the full image would be very slow on a CPU. We resize it to `512x512` for *inference*, and then we will scale the *result* back up. This is a common technique to balance speed and accuracy.

2.  **Download Model Checkpoint:** Downloads the pre-trained `Prithvi-100M-sen1floods11` model weights (`.pth` file) from the **Hugging Face Hub**. This file contains the "brain" of the model with all the learned parameters.

3.  **Build Model Architecture:** Uses `TerraTorch`'s `EncoderDecoderFactory` to construct the "empty" skeleton of the Prithvi model. We then load the downloaded weights (the "brain") into this skeleton. We also explicitly set the device to **`cpu`**.

4.  **Prepare Data:** Uses our helper functions to load, stack, normalize, and resize the 6 TIFs into a single `tensor`. A tensor is the primary data structure used by PyTorch, similar to a multi-dimensional array.

5.  **Run Inference:** This is the prediction step. We use `torch.no_grad()` and `model.eval()` to tell PyTorch we are in *inference mode* (predicting), not *training mode*. This makes the process much faster and more memory-efficient. The model's output is a `tensor` where each pixel has two values (a "score" for "Not Water" and a "score" for "Water").

6.  **Save the Result:** We take the class with the highest score (using `torch.argmax`) to create a binary mask (0 = Land, 1 = Water). We then resize this small mask back up to the original image's high resolution and save it as a new GeoTIFF file named `flood_mask.tif`.

In [ ]:
import torch
import numpy as np
import rasterio
import rasterio.warp
import os
import time
from huggingface_hub import hf_hub_download
from terratorch.models import EncoderDecoderFactory

print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print()
# --- ASCII Art changed to gray gradient ---
print("  \033[97m██████\033[37m  ██████\033[90m  ██\033[97m ████████\033[37m ██   ██\033[97m ██    ██\033[90m ██\033[0m")
print("  \033[97m██   ██\033[37m ██   ██\033[90m ██\033[97m    ██   \033[37m ██   ██\033[97m ██    ██\033[90m ██\033[0m")
print("  \033[97m██████\033[37m  ██████\033[90m  ██\033[97m    ██   \033[37m ███████\033[97m ██    ██\033[90m ██\033[0m")
print("  \033[97m██\033[37m      ██   ██\033[90m ██\033[97m    ██   \033[37m ██   ██\033[97m  ██  ██\033[90m  ██\033[0m")
print("  \033[97m██\033[37m      ██   ██\033[90m ██\033[97m    ██   \033[37m ██   ██\033[97m   ████\033[90m   ██\033[0m")
print()
print("  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print()

print("  🛰️  FLOOD DETECTION: SLIDING WINDOW INFERENCE")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

# --- 1. Helper Function (renamed to avoid MONAI conflict) ---

def predict_sliding_window_batched(model, input_tensor, window_size=512, stride=512, batch_size=4):
    """
    OPTIMIZED: Batched sliding window inference with downsampling support.
    - Processes multiple tiles simultaneously (batch_size=4 for E4s CPU)
    - Supports stride to reduce tile count
    """
    if len(input_tensor.shape) == 4:
        input_tensor = input_tensor.squeeze(0)
        
    c, h, w = input_tensor.shape
    
    # Pad to be divisible by window_size
    pad_h = (window_size - h % window_size) % window_size
    pad_w = (window_size - w % window_size) % window_size
    
    padded_input = torch.nn.functional.pad(
        input_tensor.unsqueeze(0), 
        (0, pad_w, 0, pad_h), 
        mode='reflect'
    ).squeeze(0)
    
    pad_h_total, pad_w_total = padded_input.shape[1], padded_input.shape[2]
    output_mask = torch.zeros((pad_h_total, pad_w_total), dtype=torch.uint8)
    
    # Generate all tile coordinates
    tiles = []
    for y in range(0, pad_h_total - window_size + 1, stride):
        for x in range(0, pad_w_total - window_size + 1, stride):
            tiles.append((y, x))
    
    total_tiles = len(tiles)
    print(f"      → processing {total_tiles} tiles in batches of {batch_size}")
    
    model.eval()
    with torch.no_grad():
        # Process in batches
        for batch_idx in range(0, total_tiles, batch_size):
            batch_tiles = tiles[batch_idx:batch_idx + batch_size]
            
            # Stack batch
            batch = torch.stack([
                padded_input[:, y:y+window_size, x:x+window_size] 
                for y, x in batch_tiles
            ]).to(device)
            
            # Run model on batch
            out = model(batch)
            
            # Handle output
            if hasattr(out, 'logits'):
                logits = out.logits
            elif hasattr(out, 'output'):
                logits = out.output
            else:
                logits = out
                
            preds = torch.argmax(logits, dim=1).cpu()
            
            # Place predictions
            for i, (y, x) in enumerate(batch_tiles):
                output_mask[y:y+window_size, x:x+window_size] = preds[i]
            
            # Progress
            if (batch_idx // batch_size) % 5 == 0:
                print(f"      → {batch_idx + len(batch_tiles)}/{total_tiles} tiles", end='\r')
        
        print()
                
    return output_mask[:h, :w].numpy()


def normalize_and_stack(band_files, reference_profile):
    """Reads and stacks bands, reprojects if needed."""
    bands = []
    ref_height = reference_profile['height']
    ref_width = reference_profile['width']
    
    for f in band_files:
        with rasterio.open(f) as src:
            # Check if reprojection is needed
            if (src.transform != reference_profile['transform'] or 
                src.width != ref_width or 
                src.height != ref_height):
                
                # Need to reproject
                destination = np.zeros((ref_height, ref_width), dtype=np.float32)
                source_data = src.read(1)  # Read the data first
                
                rasterio.warp.reproject(
                    source=source_data,
                    destination=destination,
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=reference_profile['transform'],
                    dst_crs=reference_profile['crs'],
                    resampling=rasterio.warp.Resampling.bilinear
                )
                bands.append(destination)
            else:
                # Can use directly
                bands.append(src.read(1).astype(np.float32))
    
    # Stack and normalize
    stacked = np.stack(bands, axis=0)
    stacked = stacked.astype(np.float32) / 10000.0  # Sentinel-2 scaling
    stacked = np.clip(stacked, 0.0, 0.3) / 0.3      # Normalize to [0, 1]
    
    return torch.from_numpy(stacked)


# --- STAGE 1: Setup --- 
print("[1/4] Setting up environment")
device = torch.device("cpu")
print(f"      → device: {device}")
print()

# --- STAGE 2: Load Model --- 
print("[2/4] Loading Prithvi model")
model_repo = "ibm-nasa-geospatial/Prithvi-100M-sen1floods11"
model_filename = "sen1floods11_Prithvi_100M.pth"

print(f"      → downloading from HuggingFace: {model_repo}")
model_checkpoint = hf_hub_download(repo_id=model_repo, filename=model_filename)

print("      → building model architecture...")
factory = EncoderDecoderFactory()
model = factory.build_model(
    task="segmentation",
    backbone="prithvi_vit_100",
    decoder="FCNDecoder",
    num_classes=2,
    backbone_kwargs={"in_channels": 6}
).to(device)

print("      → loading weights...")
checkpoint = torch.load(model_checkpoint, map_location=device)
model.load_state_dict(checkpoint.get('state_dict', checkpoint), strict=False)
print("      ✓ Model ready")
print()

# --- STAGE 3: Prepare Data (WITH DOWNSAMPLING) ---
print("[3/4] Reading and downsampling data for faster CPU inference")

# Target size for downsampling (balances speed vs quality)
TARGET_SIZE = 2048  # Reduced from ~11000 to 2048 (5x faster)

with rasterio.open(band_filepaths[0]) as src:
    ref_profile = src.profile

original_h = ref_profile['height']
original_w = ref_profile['width']
print(f"      → original size: {original_h}×{original_w} pixels")

# Calculate scale factor
scale_factor = TARGET_SIZE / max(original_h, original_w)
new_h = int(original_h * scale_factor)
new_w = int(original_w * scale_factor)
print(f"      → downsampling to: {new_h}×{new_w} pixels (~{scale_factor:.2f}x)")

# Update profile for downsampled dimensions
downsampled_profile = ref_profile.copy()
downsampled_profile.update({
    'height': new_h,
    'width': new_w,
    'transform': rasterio.transform.from_bounds(
        *rasterio.transform.array_bounds(original_h, original_w, ref_profile['transform']),
        new_w, new_h
    )
})

full_tensor = normalize_and_stack(band_filepaths, downsampled_profile)
print(f"      → final tensor shape: {full_tensor.shape}")
print("      ✓ Data loaded and downsampled")
print()

# --- STAGE 4: Inference (BATCHED) ---
print("[4/4] Running OPTIMIZED batched sliding window inference")
print("      ⏱️  Estimated time: 3-4 minutes (optimized for CPU)...")
start_time = time.time()

# Use batched inference (4 tiles at once on E4s)
downsampled_mask = predict_sliding_window_batched(
    model, 
    full_tensor, 
    window_size=512, 
    stride=512,
    batch_size=4  # Process 4 tiles simultaneously
)

elapsed = time.time() - start_time
minutes = int(elapsed // 60)
seconds = int(elapsed % 60)
print(f"      ✓ Inference complete in {minutes}m {seconds}s")

# Upscale mask back to original resolution
print(f"\n      → upscaling from {downsampled_mask.shape} to ({original_h}, {original_w})")
final_mask = np.round(
    resize(downsampled_mask, (original_h, original_w), order=0, preserve_range=True, anti_aliasing=False)
).astype(np.uint8)

# Check prediction statistics
unique, counts = np.unique(final_mask, return_counts=True)
print(f"\n      📊 Prediction Statistics:")
for cls, count in zip(unique, counts):
    percentage = (count / final_mask.size) * 100
    class_name = "Land" if cls == 0 else "Water"
    print(f"         Class {cls} ({class_name}): {count:,} pixels ({percentage:.2f}%)")
print()

# Save Result
mask_filepath = "flood_mask.tif"
out_profile = ref_profile.copy()
out_profile.update(count=1, dtype='uint8', compress='lzw')

with rasterio.open(mask_filepath, 'w', **out_profile) as dst:
    dst.write(final_mask, 1)

print(f"      → saved to: {mask_filepath}")
print(f"      → file size: {os.path.getsize(mask_filepath) / (1024*1024):.1f} MB")
print()
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print("✅ Pipeline complete - ready for COG conversion")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

In [ ]:
# Quick visualization check
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(12, 8))
im = ax.imshow(final_mask, cmap='Blues', vmin=0, vmax=1)
ax.set_title('Flood Detection Result (0=Land, 1=Water)')
plt.colorbar(im, ax=ax, label='Class')
plt.tight_layout()
plt.show()

print(f"Shape: {final_mask.shape}")
print(f"Unique values: {np.unique(final_mask)}")

# 6\. 📊 Visualize the Result

The analysis is done! We now have two key files ready for visualization:

1.  **`True_Color.jpg`**: The real satellite image preview we downloaded (though we won't use it, as `leafmap` provides its own basemaps).
2.  **`flood_mask.tif`**: The AI's prediction of where the water is (a file with 0s for land and 1s for water).


In [ ]:
import leafmap.foliumap as leafmap
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.mask import mask
import numpy as np
from PIL import Image
import os
import folium
from folium.raster_layers import ImageOverlay
import glob
import geopandas as gpd

print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print("📊 VISUALIZATION (CLIPPED TO AOI)")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

# --- 1. Helper Function: Clip Raster to AOI ---
def clip_raster_to_aoi(src_tif, aoi_gdf, out_tif):
    """Crops the raster to the AOI boundary."""
    with rasterio.open(src_tif) as src:
        # 1. Project AOI to match the Raster's CRS (e.g., UTM)
        aoi_reprojected = aoi_gdf.to_crs(src.crs)
        
        # 2. Crop the image
        # shapes expects a list of GeoJSON-like geometries
        out_image, out_transform = mask(src, aoi_reprojected.geometry, crop=True)
        out_meta = src.meta.copy()

        # 3. Update metadata (height, width, transform)
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        # 4. Save clipped file
        with rasterio.open(out_tif, "w", **out_meta) as dest:
            dest.write(out_image)
            
    return out_tif

# --- 2. Helper Function: Create Overlay (Reproject & Color) ---
def create_flood_overlay(tif_path, png_path, downscale_factor=0.2):
    if not os.path.exists(tif_path):
        return None, None

    with rasterio.open(tif_path) as src:
        # Destination CRS: Web Mercator/LatLon
        dst_crs = 'EPSG:4326'

        # Calculate dimensions
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )

        dst_width = int(width * downscale_factor)
        dst_height = int(height * downscale_factor)

        transform, _, _ = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height,
            *src.bounds, dst_width=dst_width, dst_height=dst_height
        )

        destination = np.zeros((dst_height, dst_width), dtype=np.uint8)

        reproject(
            source=rasterio.band(src, 1),
            destination=destination,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest
        )

        # Create RGBA Image
        rgba = np.zeros((dst_height, dst_width, 4), dtype=np.uint8)
        water_mask = (destination == 1)
        # Dodger Blue with opacity
        rgba[water_mask] = [30, 144, 255, 180]

        img = Image.fromarray(rgba)
        img.save(png_path)

        b = rasterio.transform.array_bounds(dst_height, dst_width, transform)
        # Folium bounds: [[lat_min, lon_min], [lat_max, lon_max]]
        bounds = [[b[1], b[0]], [b[3], b[2]]]

        return png_path, bounds

# --- 3. Main Workflow ---
print("[1/3] finding and processing data...")

# Find input file
possible_files = ["flood_mask (1).tif", "flood_mask.tif"]
input_tif = next((f for f in possible_files if os.path.exists(f)), None)

if not input_tif:
    # Fallback search
    tiffs = glob.glob("*.tif")
    if tiffs: input_tif = tiffs[0]

if input_tif:
    print(f"      → Input: {input_tif}")
    
    # NEW STEP: Clip to AOI if available
    processing_tif = input_tif
    if 'aoi_gdf' in locals():
        print("      → Clipping raster to AOI boundary...")
        clipped_tif = "flood_mask_clipped.tif"
        try:
            processing_tif = clip_raster_to_aoi(input_tif, aoi_gdf, clipped_tif)
            print("      ✓ Clip successful")
        except Exception as e:
            print(f"      ⚠️ Clip failed ({e}), using full image instead.")
    
    # Generate Overlay
    overlay_png = "flood_overlay_final.png"
    img_path, img_bounds = create_flood_overlay(processing_tif, overlay_png)

    if img_path:
        # --- 4. Initialize Map ---
        print("[2/3] Initializing map...")
        
        # Center on the AOI
        if 'aoi_gdf' in locals():
            aoi_center = aoi_gdf.geometry.iloc[0].centroid
            center_lat, center_lon = aoi_center.y, aoi_center.x
        else:
            center_lat = (img_bounds[0][0] + img_bounds[1][0]) / 2
            center_lon = (img_bounds[0][1] + img_bounds[1][1]) / 2

        m = leafmap.Map(center=(center_lat, center_lon), zoom=13, height="600px")
        m.add_basemap("OpenStreetMap")
        m.add_basemap("Esri.WorldImagery")

        # --- 5. Add Overlay ---
        print("[3/3] Adding overlay...")
        
        overlay = ImageOverlay(
            name="AI Flood Prediction",
            image=overlay_png,
            bounds=img_bounds,
            opacity=0.8,
            interactive=True,
            cross_origin=False,
            zindex=1
        )
        overlay.add_to(m)

        if 'aoi_gdf' in locals():
            folium.GeoJson(
                data=aoi_gdf,
                name="Study Area (AOI)",
                style_function=lambda x: {'color': 'red', 'fillColor': 'transparent', 'weight': 2, 'dashArray': '5, 5'}
            ).add_to(m)

        folium.LayerControl(collapsed=False).add_to(m)
        print("      ✓ Visualization ready.")
    else:
        print("❌ Error processing overlay.")
else:
    print("❌ Input TIF file not found.")

m

### 🗺️ Analyzing the Output

Look at the map! The interactive map above displays the final result of our workflow. (You can use the layer control in the top-right corner to toggle layers on and off).

  * **OpenStreetMap (Basemap):** This provides the context of roads and city names, like **Vicksburg**.
  * **AI Flood Detection (Blue Overlay):** This is our `flood_mask_cog.tif` file. The Prithvi model has generated this layer. The blue areas represent all the pixels that the AI classified as "Water" (class 1).
  * **Study Area (Red Box):** This is the AOI we defined back in Step 3.

As you can see from the provided sample output, the **blue overlay** aligns *extremely* well with the river channels visible in the satellite imagery (and on the basemap). Notice how it's not just a rough blob; the model has captured the precise, complex shape of the **Mississippi River** and the **Yazoo River** to the north. It correctly identified the main channels, smaller tributaries, and even the "cut-off" oxbow lakes in the floodplain. This demonstrates the model's high level of accuracy in segmenting water from land, even in a complex riverine environment.

## 7. 🔬 Validating the Result (Ground Truth Check)

Our model produced a prediction, but how do we know it's accurate? This final step, **validation**, is one of the most important parts of any AI workflow. We need to compare our result to a "ground truth" to confirm its real-world value.

We can do this in two simple ways:

#### Method 1: Visual Sanity Check (vs. Basemap)
Use the layer control (top right) in your interactive map to toggle the "AI Flood Detection" layer on and off. Switch the basemap from "OpenStreetMap" to "Satellite" (often called `Esri.WorldImagery` or similar) to compare satellite-to-satellite.

**Observation 1 (Accuracy):** You should see that the blue overlay perfectly aligns with the permanent river channels (the Mississippi and Yazoo) visible on the satellite basemap. This confirms the model isn't "hallucinating" and has a very high spatial accuracy.

**Observation 2 (Flooding):** Look closely at the areas outside the main channel, especially near the "Fort of Vicksburg" and inside the river bends (oxbows). The blue mask extends beyond the main channel into low-lying floodplain areas. This is the model's "flood" detection.

#### Method 2: Real-World Event Verification
The visual check strongly suggests flooding, but was there *actually* a flood in Vicksburg when this image was taken (April-May 2023)? This is the "ground truth" check.

A quick search for "Vicksburg MS flooding April 2023" confirms our data.

**Ground Truth:** The National Weather Service (NWS) reported that the Mississippi River at Vicksburg was in **major flood stage** throughout April and early May 2023. The river crested at over 48 feet (well above the 43-foot flood stage), inundating thousands of acres of surrounding low-lying farmland and floodplain areas—exactly what the model detected.

---

### 🏁 Conclusion
Our workflow is validated. The `Prithvi-100M-sen1floods11` model:

*   Correctly identified all permanent water.
*   Detected additional surface water in known floodplains.
*   This detection matches ground-truth reports of a major, real-world flood event.

This confirms that the model is not just guessing—it's accurately segmenting a real, ongoing event.

# 🚀 Exploration (Optional Next Steps)

Congratulations on completing the lab! You've successfully run a state-of-the-art Geospatial Foundation Model. If you have extra time, try these challenges to build on what you've learned.

-----

### 1\. Change the AOI

Go back to **Step 3 (Cell 9)** and change the `min_lon, min_lat, max_lon, max_lat` coordinates.

  * **Idea:** Try a large inland lake, like Lake Okeechobee in Florida.
  * *Run* all the cells again and see if the model still works.

<!-- end list -->

```python
# Challenge: Find Lake Okeechobee
# Hint: It's around 26.9° N, 80.8° W
min_lon, min_lat, max_lon, max_lat = [____, 26.7, ____, 27.1]
```
### 2\. Try a Different Model (Burn Scar Detection)

This workflow isn't just for floods — you can swap in a **fire burn-scar detection** model.

### 👉 How to do it
Go to **Step 5 (Cell 14)** and change the model:

- Replace the model repo with:  
  `ibm-nasa-geospatial/Prithvi-100M-burn-scar`
- On the Hugging Face page, find the correct **checkpoint filename** (the `.pth` file)
- Pick a new **AOI over a wildfire region** (California, Australia, etc.)
- Update the visualization colormap in **Step 6 (Cell 18)**  
  from `"Blues"` → `"Reds"` or `"OrRd"` since fire burn scars are reddish/brown

### 🔧 Code to Paste (Step 5 modification)

```python
# Challenge: Modify Step 5 for the Burn Scar Model
model_repo = "ibm-nasa-geospatial/Prithvi-100M-burn-scar"
model_checkpoint = hf_hub_download(
    repo_id=model_repo,
    filename="______.pth"  # <-- Find the correct filename on Hugging Face
)

# Modify Step 6 for the Burn Scar Model
m.add_raster(
    "flood_mask_cog.tif", # You should rename this to "burn_mask_cog.tif"
    colormap="____", # <-- Use a color that makes sense for fire (e.g., "Reds")
    nodata=0,
    layer_name="AI Burn Scar Detection",
    opacity=0.7
)

```
### 3. Compare to a Traditional Index (NDWI)

In traditional remote sensing, we use simple spectral indices to highlight features.  
One classic example is **NDWI — Normalized Difference Water Index**.

**Formula:**  
\[
\text{NDWI} = \frac{Green - NIR}{Green + NIR}
\]

**How it works:**  
- Load **Green.tif** (Sentinel-2 Band 3) and **Nir.tif** (Band 8) using `rasterio`
- Perform the NDWI formula
- Save the resulting raster to a GeoTIFF
- Optionally add it to your Leafmap display in Step 6  
- Then compare:  
  - Where does NDWI match the AI flood prediction?  
  - Where does the AI model detect water that NDWI misses?  
  - Where is NDWI noisy or incorrect?

---

### NDWI Calculation Code (add as a new cell after Step 4)

```python
# Challenge: Calculate NDWI (add a new cell after Step 4)
print("Calculating NDWI...")

# 1. Define file paths
green_path = os.path.join(data_dir, "____.tif")  # <-- Green band filename (e.g., "Green.tif")
nir_path = os.path.join(data_dir, "____.tif")    # <-- NIR band filename (e.g., "Nir.tif")
ndwi_path = "ndwi_result.tif"

# 2. Open files
with rasterio.open(green_path) as green_src:
    green = green_src.read(1).astype("float32")
    profile = green_src.profile

with rasterio.open(nir_path) as nir_src:
    nir = nir_src.read(1).astype("float32")

# 3. Calculate NDWI (with a check for division by zero)
# Sentinel-2 reflectance is typically scaled by 10,000 — float32 makes this safe
numerator = green - nir
denominator = green + nir

# Avoid division-by-zero
ndwi = np.where(denominator == 0, 0, numerator / denominator)

# 4. Save the result
profile.update(dtype="float32", count=1, compress="lzw")

with rasterio.open(ndwi_path, "w", **profile) as dst:
    dst.write(ndwi, 1)

print(f"✓ NDWI calculation complete: {ndwi_path}")

# 5. (Optional) Add this new file to your leafmap in Step 6!
# m.add_raster(ndwi_path, colormap="RdBu", layer_name="NDWI")
```